<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install optuna

In [ ]:
import optuna
import matplotlib.pyplot as plt
import numpy as np

import random

In [ ]:
study = optuna.create_study(storage='sqlite:///model_selection.db', study_name='model_selection', direction='minimize', load_if_exists=True)

In [ ]:
def f(x, y, z):
    return x**2 + y**2 + z**2

In [ ]:
class MyException(Exception):
    pass

def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    y = trial.suggest_uniform('y', -10, 10)
    z = trial.suggest_uniform('z', -10, 10)
    # if random.random() < 0.1:
    #     raise MyException()
    return f(x, y, z)

In [ ]:
study.optimize(objective, n_trials=100, catch=(MyException,))

In [ ]:
study.best_params

In [ ]:
study.best_trial

# Visualizzazione

In [ ]:
study = optuna.load_study(study_name='model_selection', storage='sqlite:///model_selection.db' )

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_edf(study)

# pruning

In [ ]:
def random_walk(prob_left=0.5, step=1, start_value=100, steps=100):
    prob_right = 1 - prob_left
    value = start_value
    for _ in range(steps):
        delta = random.choices(population=[-step, step], weights=[prob_left, prob_right], k=1)[0]
        value += delta
        yield value

In [ ]:
def objective(trial):
        prob_left = trial.suggest_uniform('prob_left', 0, 1)
        step = trial.suggest_uniform('step', 0, 1)
        generator = random_walk(prob_left=prob_left, step=step)
        for step, intermediate_value in enumerate(generator):
            trial.report(intermediate_value, step)
            if trial.should_prune():
                raise optuna.TrialPruned()

        final_value = intermediate_value
        return final_value

In [ ]:
study = optuna.create_study(storage='sqlite:///pruner.db', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=100, n_jobs=-1)